A loja vende mais no verão?

In [ ]:
from google.colab import drive 
from scipy.stats import zscore
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:
drive.mount('/content/drive')

# Coleta dos dados

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/compras_loja_praia.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

No dataset, a coluna idade do cliente chama atenção porque o desvio padrão é quase o dobro da média. Isso indica uma dispersão muito alta e sugere a presença de outliers. Além disso, apenas cerca de 49% dos dados estão preenchidos, o que limita bastante as análises com essa variável e pode exigir tratamento ou até exclusão da coluna, dependendo do objetivo.

Na coluna quantidade, todos os valores estão preenchidos. A média é 39, mas o desvio padrão é 446 — mais de 10 vezes maior. Essa diferença é muito grande e praticamente confirma a existência de outliers. Porém, os quartis parecem consistentes.

Já na coluna preço unitário, os dados estão completos e, no geral, parecem coerentes. O único ponto de atenção é o valor mínimo de 0,01, que pode indicar erro de registro.

# Observação dos dados

In [ ]:
df.isna().sum()

Como nas colunas "cidade" e "idade_cliente" está faltando mais de 50% dos dados, não será possícel utilizá-las para a análise.

In [ ]:
df.duplicated().sum()

### Idade

In [ ]:
df[["idade_cliente"]].hist(bins = 100, figsize=(15,5))
plt.suptitle("Distribuição idade")
plt.show()

In [ ]:
sns.boxplot(x = df["idade_cliente"])
plt.suptitle("Distribuição idade")
plt.show()

O boxplot foi criado para poder analisar os outilers e a sua quantidade. Como podemos ver no gráfico, apenas um outlier existe, no valor de quase 1000 anos.

### Preço unitário

In [ ]:
df[["preco_unitario"]].hist(bins = 100, figsize=(15,5))
plt.suptitle("Distribuição preço")
plt.show()

Pelo histograma, podemos ver que temos quase 20 discrepâncias (valor sem sentido) com o valor de, aproximadamente, 0.

### Quantidade

In [ ]:
df[["quantidade"]].hist(bins = 10, figsize=(15,5))
plt.suptitle("Distribuição quantidade")
plt.show()

In [ ]:
sns.boxplot(x = df["quantidade"])
plt.suptitle("Distribuição quantidade")
plt.show()

Pelo boxplot, conseguimos ver que essa coluna está com dados bem distorcidos. Porém, possui apenas 2 valores de outliers.

# Tratamento

* Arrumar data 
* Replace de gênero 
* Tratar outlier de quantidade
* Tratar discrepância de preço unitário
* Tratar duplicatas

### Arrumar data

In [ ]:
df["data_compra_corrigida"] = pd.to_datetime(df["data_compra"], errors="coerce") # Corrigir tipo do campo data

In [ ]:
df.info()

Podemos ver que a nova coluna "data_compra_corrigida" foi criada com sucesso e é um datetime. Porém, ela possui 20 valores a menos que a coluna original. Como é menos de 1% da quantidade total, não há problemas.

### Replace de gênero

In [ ]:
df["genero_cliente"] = df["genero_cliente"].fillna("Não informado") # Preenchimento de gêneros ausentes com "Não informado"

### Outlier quantidade

In [ ]:
# Calculo de outlier usando IQR
Q1 = df['quantidade'].quantile(0.25)
Q3 = df['quantidade'].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# O que estiver abaixo ou acima de algum imite, é considerado outlier

outliers_iqr = df[(df['quantidade'] < limite_inferior) | (df['quantidade'] > limite_superior)]
outliers_iqr

Com o IQR, temos 17% de outliers, valor considerado extremamente alto. Isso sugere que ele não está calculando da maneira mais adequadas para esse dataset e, com isso, calcularemos de outra forma usando o z-score.

In [ ]:
# Calculo de outliers com z-score
df["quantidade_zscore"] = zscore(df['quantidade'])
outliers_z = df.copy()
outliers_z = outliers_z[outliers_z["quantidade_zscore"] > 3]
outliers_z.describe()

In [ ]:
# Retirar outliers de quantidade
df = df[df["quantidade_zscore"] < 3]
df.describe()

### Discrepância de preço unitário

In [ ]:
# Verificar discrepância
df_zoom_25 = df.copy()
df_zoom_25 = df_zoom_25[df_zoom_25["preco_unitario"] < 1]


df_zoom_25[["preco_unitario"]].hist(bins = 100, figsize=(15,5))
plt.suptitle("Distribuição preço")
plt.show()

Como são 15 unidades que possuem essa discrepância, poderíamos apenas removê-las, já que seriam menos de 0,3% do total. Porém, outra forma de tratá-las seria com o replace.

In [ ]:
# Substituição
Q1_preco = df['preco_unitario'].quantile(0.25)
df["preco_unitario"] = df["preco_unitario"].replace(0.01, Q1)
df.describe()

### Limpar ausentes da data

In [23]:
df = df.dropna(subset=["data_compra_corrigida"])

# EDA


* Vende mais no verão? sim
* O que muda do verão para as outras estações? os clientes compram em maior quantidade

In [30]:
# Gerar df tratado apenas com colunas úteis
df_tratado = df[['id_compra', 'cliente_id',
        'produto', 'quantidade', 'preco_unitario',
       'data_compra_corrigida']]

In [ ]:
# Criar colunas de mês e ano
df_tratado["mes"] = df_tratado["data_compra_corrigida"].dt.month
df_tratado["ano"] = df_tratado["data_compra_corrigida"].dt.year
df_tratado

In [ ]:
# Criar coluna de estação com base no mês do hemisfério sul
df_tratado["estacao"] = df_tratado["mes"].map({1: "Verão", 2: "Verão", 3: "Outono", 4: "Outono", 5: "Outono", 6: "Inverno", 7: "Inverno", 8: "Inverno", 9: "Primavera", 10: "Primavera", 11: "Primavera", 12: "Verão"})


In [ ]:
# Criar coluna do total vendido para análise
df_tratado["valor_total"] = df_tratado["quantidade"] * df_tratado["preco_unitario"]
df_tratado.head()

In [ ]:
# Análise de valor vendido por estação 
vendas_estacao = df_tratado.groupby("estacao")["valor_total"].sum()

In [ ]:
# Análise de quantidade de compradores por estação
clientes_estacao = df_tratado.groupby("estacao")["cliente_id"].nunique()
clientes_estacao

,cliente_id
estacao,
Inverno,2473
Outono,2538
Primavera,2475
Verão,2474


Como podemos ver, a quantidade de clientes nas diferentes estações praticamente se mantém constante. Isso nos leva a pensar que é o ticket médio que aumenta no verão.

In [43]:
# Análise de ticket médio por estação
ticket_estacao = df_tratado.groupby("estacao")["valor_total"].mean()
ticket_estacao

,valor_total
estacao,
Inverno,2004.112312
Outono,2079.102105
Primavera,2061.032694
Verão,2173.646770


In [ ]:
# Análise de quantidade de itens comprados por estação
quantidade_estacao = df_tratado.groupby("estacao")["quantidade"].mean()
quantidade_estacao

,quantidade
estacao,
Inverno,19.594290
Outono,19.309505
Primavera,19.004014
Verão,20.096943


In [45]:
# Análise do preço unitário por estação
preco_estacao = df_tratado.groupby("estacao")["preco_unitario"].mean()
preco_estacao

,preco_unitario
estacao,
Inverno,107.411749
Outono,108.913940
Primavera,104.959177
Verão,107.869308


Chegamos a conclusão que no verão há mais lucro, porém não é porque existem mais clientes comprando ou porque estão comprando itenis mais caros, e sim porque a média de compras por cliente aumenta.